# 1. Data Analysis of the Influencers:
# 2. Sentiment Analysis of the Influencers: 


In [1]:
# Import the Dependencies
import tweepy
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
# import time
from datetime import datetime
# import seaborn as sns
from os import path, makedirs # fetch path and makedirs function from os file
import csv # fetch csv file
from glob import glob # fetching glob function only from the glob lib

In [2]:
# Import keys from the config file
from config import consumer_key, consumer_secret, access_token, access_token_secret

In [3]:
# # Twitter API Keys
# consumer_key = 'Your Key'
# consumer_secret = 'Your Key'
# access_token = 'Your Key'
# access_token_secret = 'Your Key'

In [4]:
# Import and Initialize Sentiment Analyzer
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
analyzer = SentimentIntensityAnalyzer()

In [5]:
# Setup Tweepy API Authentication
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth, parser=tweepy.parsers.JSONParser())

In [6]:
# Load and read csv file containing the details of the Influencers 
influencer_data_load = "SentimentInfluencerInputDataIB_3.csv"
influencer_data_read = pd.read_csv(influencer_data_load)

In [ ]:
# ### Test Cell
# ## 'entities': {'hashtags': [],
# target_user = '@Riicha_r'
# Riicha_account = api.get_user(target_user)
# # Retweet = user_account['status']['extended_entities'] ['retweet_count']
# Retweet = Riicha_account['status'] ['retweet_count']
# # datetime.strptime(user_account["created_at"],'%m/%d/%Y' )
# datetime.strptime(Riicha_account['created_at'],'%A %B %d %H:%M %Y')
# # Thu Jul 16 04:02:35 +0000 2009
# Samp datetime_object = datetime.strptime('Jun 1 2005  1:33PM', '%b %d %Y %I:%M%p')

In [7]:
# for index, row in Influencers_DF.iterrows():
Data_Influencers_DF = influencer_data_read
Not_Found = 0
print("-----------Start extraction!!!-----------")

for index, row in Data_Influencers_DF.iterrows():
    target_user = row["Twitter_Handle"]
    
    try:
        user_account = api.get_user(target_user)
        # user_account.keys()
        
        # user_real_name = user_account["name"]
        # F_name = row["First_Name"]
        user_geo_enabled = user_account["geo_enabled"]
        if (user_geo_enabled == True):
            user_location = user_account["location"]
            Data_Influencers_DF.at[index, "Loc"] = user_location
        else:
            Data_Influencers_DF.at[index, "Loc"] = 'NA'
#         lang = 'NA'
               
        if (user_account["lang"] == 'en'):
            user_Lang = user_account["lang"]
            Data_Influencers_DF.at[index, "Lang"] = 'Eng'
        else:
            Data_Influencers_DF.at[index, "Lang"] = 'NA'
                
#         Data_Influencers_DF.at[index, "Total Tweets"] = user_account["statuses_count"] 
#         Data_Influencers_DF.at[index, "Retweets"] = user_account['status'] ['retweet_count']
    
        
# # Tweet_Count =  statuses_count - retweet_count
# #         Data_Influencers_DF.at[index, "Tweets Only"] = int(user_account["statuses_count"]) - int(user_account['status']['retweet_count'])  
        Data_Influencers_DF.at[index, "Created On"] = user_account["created_at"]

#         Data_Influencers_DF.at[index, "Following"] = user_account["following"]  

# # 'retweet_count':                       
        
        user_tweets = user_account["statuses_count"]
        user_followers = user_account["followers_count"]
        user_following = user_account["friends_count"]
        user_favorites = user_account["favourites_count"]
        
       #  Data_Influencers_DF.at[index, "Real Name"] = user_real_name
        Data_Influencers_DF.at[index, "Tweets"] = user_tweets
        Data_Influencers_DF.at[index, "Followers"] = user_followers
        Data_Influencers_DF.at[index, "Following"] = user_following
        Data_Influencers_DF.at[index, "Favorites Count"] = user_favorites
        
              
    
    except tweepy.TweepError as e:
        Not_Found = Not_Found + 1
        print(f"exception for {row['Twitter_Handle']}: {e}")

print("----------- Extraction Complete !!!-----------")        
print(Not_Found)


-----------Start extraction!!!-----------
exception for @Relando_T: [{'code': 50, 'message': 'User not found.'}]
----------- Extraction Complete !!!-----------
1


In [ ]:
###IB changed code - to re-index the result dataframe

# Remove missing values.
#Data_Influencers_DF = Data_Influencers_DF.reset_index(drop=True)
Data_Influencers_DF = Data_Influencers_DF.dropna()

In [ ]:
# Grouped on Genre
Data_Influencers_DF.groupby(["Genre"]).count()
#Data_Influencers_DF.sort_values(['Followers'], axis=0, ascending=False, inplace=False, kind='quicksort', na_position='last')

In [9]:
###  IB String search - added 06/29/2018

text_data_df = pd.DataFrame(Data_Influencers_DF)
text_data_df.head(10)
text_data_df.count()
### 358 good

public_twitter_text = []
public_tweet_count = 0

Not_Found_IB = 0
print("-----------Start extraction!!!-----------")

for index, row in text_data_df.iterrows():
    target_user = row["Twitter_Handle"]
    
    # Search for Celeb Tweeting about search term"
    search_term = "drug"  ### later change for dictionary of lists that explore each search term
                          ### for drug obuse: "drug addiction", "drug habbit", "rehab", etc.   
    # Search for our Influencers' tweets and see if they tweet about our "search term"
    public_tweets = api.search(search_term, count=10, result_type="recent")
    #### maybe later public_tweets = api.search(target_user, count=50, result_type="recent")
    try:
        # Loop through all public_tweets
        for tweet in public_tweets["statuses"]:

            # Print the username
            print(tweet["user"]["screen_name"])

            # Print the tweet text
            print(tweet["text"])
            print()
            
        # Print total number of tweets
        print(len(public_tweets["statuses"]))
###########################################################  
    
#     # Search for our Influencers' tweets and see if they tweet about our "search term"
#     public_tweets = api.search(target_user, count=5, result_type="recent")
#     #### maybe later public_tweets = api.search(target_user, count=100, result_type="recent")
#     try:
#         # Loop through all public_tweets
#         for tweet in public_tweets["statuses"]:

#             # Get ID and Author of most recent tweet directed to me
#             tweet_id = tweet["id"]
#             ### maybe later tweet_count = tweet[count]
#             tweet_author = tweet["user"]["screen_name"]
#             tweet_text = tweet["text"]

#             # Print Tweet Text and Tweet Author
#             ### maybe later print(tweet_count)
#             print(tweet_text)
#             print(tweet_author)
#             print(f"public_tweet_count = '{public_tweet_count}'")
  
    except tweepy.TweepError as e:
        Not_Found_IB = Not_Found_IB + 1
        print(f"exception for {row['Twitter_Handle']}: {e}")
    
print("----------- Extraction Complete !!!-----------")        
print(Not_Found)
###########################################################
#         # user_account.keys()
        
#         # user_real_name = user_account["name"]
#         # F_name = row["First_Name"]
#         user_geo_enabled = user_account["geo_enabled"]
#         if (user_geo_enabled == True):
#             user_location = user_account["location"]
#             Data_Influencers_DF.at[index, "Loc"] = user_location
#         else:
#             Data_Influencers_DF.at[index, "Loc"] = 'NA'
# #         lang = 'NA'
               
#         if (user_account["lang"] == 'en'):
#             user_Lang = user_account["lang"]
#             Data_Influencers_DF.at[index, "Lang"] = 'Eng'
#         else:
#             Data_Influencers_DF.at[index, "Lang"] = 'NA'
                
# #         Data_Influencers_DF.at[index, "Total Tweets"] = user_account["statuses_count"] 
# #         Data_Influencers_DF.at[index, "Retweets"] = user_account['status'] ['retweet_count']


-----------Start extraction!!!-----------
Thee_Doggler
@IgnantVegeta good drug though https://t.co/yIi0bp3Eds

bc_516
RT @MrQSharp: only drug i take tbh https://t.co/m3JPQZiFj7

CaryNovak
RT @Barnes_Law: #Mueller covered up the two great #DeepState dirty money banks: BCCI in the 1990s (key Democrats, like former AG Clifford,…

ambarro
RT @MattieRheumMD: Thank you @SecAzar- He "blames PBMs for ‘broken system’ in Senate Finance hearing" and said that if pharma co. lowered i…

ABatemanHouse
RT @abettel: Drug and medical device companies made $8.4 billion in payments to physicians and teaching hospitals in 2017, a slight drop fr…

chelley1956
RT @Barnes_Law: #Mueller covered up the two great #DeepState dirty money banks: BCCI in the 1990s (key Democrats, like former AG Clifford,…

indiessexuaI
drug drug drug
mina se me deixa bem bem bem

AimHighMC130E
RT @cebudailynews: The drug enforcement Unit of Minglanillla police station arrested two men after a buy-bust operation on Thursday evening…

Thee_Doggler
@IgnantVegeta good drug though https://t.co/yIi0bp3Eds

bc_516
RT @MrQSharp: only drug i take tbh https://t.co/m3JPQZiFj7

CaryNovak
RT @Barnes_Law: #Mueller covered up the two great #DeepState dirty money banks: BCCI in the 1990s (key Democrats, like former AG Clifford,…

ambarro
RT @MattieRheumMD: Thank you @SecAzar- He "blames PBMs for ‘broken system’ in Senate Finance hearing" and said that if pharma co. lowered i…

ABatemanHouse
RT @abettel: Drug and medical device companies made $8.4 billion in payments to physicians and teaching hospitals in 2017, a slight drop fr…

chelley1956
RT @Barnes_Law: #Mueller covered up the two great #DeepState dirty money banks: BCCI in the 1990s (key Democrats, like former AG Clifford,…

indiessexuaI
drug drug drug
mina se me deixa bem bem bem

AimHighMC130E
RT @cebudailynews: The drug enforcement Unit of Minglanillla police station arrested two men after a buy-bust operation on Thursday evening…

LZfoBHelhqF2tYm
@drug_na_palit ภาพสวยมาก

Thee_Doggler
@IgnantVegeta good drug though https://t.co/yIi0bp3Eds

bc_516
RT @MrQSharp: only drug i take tbh https://t.co/m3JPQZiFj7

CaryNovak
RT @Barnes_Law: #Mueller covered up the two great #DeepState dirty money banks: BCCI in the 1990s (key Democrats, like former AG Clifford,…

ambarro
RT @MattieRheumMD: Thank you @SecAzar- He "blames PBMs for ‘broken system’ in Senate Finance hearing" and said that if pharma co. lowered i…

ABatemanHouse
RT @abettel: Drug and medical device companies made $8.4 billion in payments to physicians and teaching hospitals in 2017, a slight drop fr…

chelley1956
RT @Barnes_Law: #Mueller covered up the two great #DeepState dirty money banks: BCCI in the 1990s (key Democrats, like former AG Clifford,…

indiessexuaI
drug drug drug
mina se me deixa bem bem bem

AimHighMC130E
RT @cebudailynews: The drug enforcement Unit of Minglanillla police station arrested two men after a buy-bust operation on Thursday evening…

LZfoBHelhqF2tYm
@drug_na_palit ภาพสวยมาก

Thee_Doggler
@IgnantVegeta good drug though https://t.co/yIi0bp3Eds

bc_516
RT @MrQSharp: only drug i take tbh https://t.co/m3JPQZiFj7

CaryNovak
RT @Barnes_Law: #Mueller covered up the two great #DeepState dirty money banks: BCCI in the 1990s (key Democrats, like former AG Clifford,…

ambarro
RT @MattieRheumMD: Thank you @SecAzar- He "blames PBMs for ‘broken system’ in Senate Finance hearing" and said that if pharma co. lowered i…

ABatemanHouse
RT @abettel: Drug and medical device companies made $8.4 billion in payments to physicians and teaching hospitals in 2017, a slight drop fr…

chelley1956
RT @Barnes_Law: #Mueller covered up the two great #DeepState dirty money banks: BCCI in the 1990s (key Democrats, like former AG Clifford,…

indiessexuaI
drug drug drug
mina se me deixa bem bem bem

AimHighMC130E
RT @cebudailynews: The drug enforcement Unit of Minglanillla police station arrested two men after a buy-bust operation on Thursday evening…

LZfoBHelhqF2tYm
@drug_na_palit ภาพสวยมาก

Thee_Doggler
@IgnantVegeta good drug though https://t.co/yIi0bp3Eds

bc_516
RT @MrQSharp: only drug i take tbh https://t.co/m3JPQZiFj7

CaryNovak
RT @Barnes_Law: #Mueller covered up the two great #DeepState dirty money banks: BCCI in the 1990s (key Democrats, like former AG Clifford,…

ambarro
RT @MattieRheumMD: Thank you @SecAzar- He "blames PBMs for ‘broken system’ in Senate Finance hearing" and said that if pharma co. lowered i…

ABatemanHouse
RT @abettel: Drug and medical device companies made $8.4 billion in payments to physicians and teaching hospitals in 2017, a slight drop fr…

chelley1956
RT @Barnes_Law: #Mueller covered up the two great #DeepState dirty money banks: BCCI in the 1990s (key Democrats, like former AG Clifford,…

indiessexuaI
drug drug drug
mina se me deixa bem bem bem

AimHighMC130E
RT @cebudailynews: The drug enforcement Unit of Minglanillla police station arrested two men after a buy-bust operation on Thursday evening…

LZfoBHelhqF2tYm
@drug_na_palit ภาพสวยมาก

...

Remainingareal1
Obviously they need to do this to pay for the Obese, drug invested and trans ops

NHS England to stop 'ineffective'… https://t.co/sVND18JTZb

ajaysapra
RT @AiyyashBilla_: Can't understand the point of making a movie on a drug addict star-kid who is serving time in jail for aiding in Bombay…

SquirrelyGrrl
@JoccoJohnson @CBSNews Pretty much any drug other than weed. In dact, weed may be related to even fewer acts of vio… https://t.co/bq4SKJfBVm

jillfahmyyahoo1
RT @Barnes_Law: #Mueller covered up the two great #DeepState dirty money banks: BCCI in the 1990s (key Democrats, like former AG Clifford,…

JackieWillard2
RT @jacobsoboroff: Turns out, drugs *aren't* pouring in where there's no wall.

They come through *legal* crossings.

We went to the Otay M…

EdnaEdnapunda
RT @WeSupport45: The Media is calling Mexico Presidential Candidate @lopezobrador_ the @realDonaldTrump of Mexico
This is a LIE!
If this co…

Saukel
Medicare spent $2 billion for one drug as the manufacturer

Remainingareal1
Obviously they need to do this to pay for the Obese, drug invested and trans ops

NHS England to stop 'ineffective'… https://t.co/sVND18JTZb

ajaysapra
RT @AiyyashBilla_: Can't understand the point of making a movie on a drug addict star-kid who is serving time in jail for aiding in Bombay…

SquirrelyGrrl
@JoccoJohnson @CBSNews Pretty much any drug other than weed. In dact, weed may be related to even fewer acts of vio… https://t.co/bq4SKJfBVm

jillfahmyyahoo1
RT @Barnes_Law: #Mueller covered up the two great #DeepState dirty money banks: BCCI in the 1990s (key Democrats, like former AG Clifford,…

JackieWillard2
RT @jacobsoboroff: Turns out, drugs *aren't* pouring in where there's no wall.

They come through *legal* crossings.

We went to the Otay M…

EdnaEdnapunda
RT @WeSupport45: The Media is calling Mexico Presidential Candidate @lopezobrador_ the @realDonaldTrump of Mexico
This is a LIE!
If this co…

Saukel
Medicare spent $2 billion for one drug as the manufacturer

Remainingareal1
Obviously they need to do this to pay for the Obese, drug invested and trans ops

NHS England to stop 'ineffective'… https://t.co/sVND18JTZb

ajaysapra
RT @AiyyashBilla_: Can't understand the point of making a movie on a drug addict star-kid who is serving time in jail for aiding in Bombay…

SquirrelyGrrl
@JoccoJohnson @CBSNews Pretty much any drug other than weed. In dact, weed may be related to even fewer acts of vio… https://t.co/bq4SKJfBVm

jillfahmyyahoo1
RT @Barnes_Law: #Mueller covered up the two great #DeepState dirty money banks: BCCI in the 1990s (key Democrats, like former AG Clifford,…

JackieWillard2
RT @jacobsoboroff: Turns out, drugs *aren't* pouring in where there's no wall.

They come through *legal* crossings.

We went to the Otay M…

EdnaEdnapunda
RT @WeSupport45: The Media is calling Mexico Presidential Candidate @lopezobrador_ the @realDonaldTrump of Mexico
This is a LIE!
If this co…

Saukel
Medicare spent $2 billion for one drug as the manufacturer

Remainingareal1
Obviously they need to do this to pay for the Obese, drug invested and trans ops

NHS England to stop 'ineffective'… https://t.co/sVND18JTZb

ajaysapra
RT @AiyyashBilla_: Can't understand the point of making a movie on a drug addict star-kid who is serving time in jail for aiding in Bombay…

SquirrelyGrrl
@JoccoJohnson @CBSNews Pretty much any drug other than weed. In dact, weed may be related to even fewer acts of vio… https://t.co/bq4SKJfBVm

jillfahmyyahoo1
RT @Barnes_Law: #Mueller covered up the two great #DeepState dirty money banks: BCCI in the 1990s (key Democrats, like former AG Clifford,…

JackieWillard2
RT @jacobsoboroff: Turns out, drugs *aren't* pouring in where there's no wall.

They come through *legal* crossings.

We went to the Otay M…

EdnaEdnapunda
RT @WeSupport45: The Media is calling Mexico Presidential Candidate @lopezobrador_ the @realDonaldTrump of Mexico
This is a LIE!
If this co…

Saukel
Medicare spent $2 billion for one drug as the manufacturer

RateLimitError: [{'message': 'Rate limit exceeded', 'code': 88}]

# 2. Sentiment Analysis of the Influencers: 

In [ ]:
###
# Influencers = influencer_data_read
# s = Influencers.iloc[:,0]


In [ ]:
# # Extract the first 100 tweets of the Influencers
# print("-----------Start extraction of the tweets posted by the Influencers!!!-----------")
# Influencers = []
# Influencers = Data_Influencers_DF
# Influencers = influencer_data_read
# # Influencers_th = Influencers.iloc[:,0]
# # # Array to hold the sentiments
# # Tweet_array = []
# Sentiment_array = []

# # for user in Influencers[0]:
# # for user in Influencers_th:
# for user in Influencers:
#     # Set the tweet count to 100
#     tweet_count = 100
#     print("Extracting tweets from %s"%user)
    
#     # Extract tweets up to 5 pages
#     for x in range(1):
#         influencer_tweets = api.user_timeline(user,page = 1)       
        
# #         # For each tweet in a bunch of public tweets
# for tweet in influencer_tweets:
            
#             #Calculate the compound, positive, negative and neutral values of each tweet
#             compound = analyzer.polarity_scores(tweet["text"])["compound"]
#             pos = analyzer.polarity_scores(tweet["text"])["pos"]
#             neu = analyzer.polarity_scores(tweet["text"])["neu"]
#             neg = analyzer.polarity_scores(tweet["text"])["neg"]
            
#             # Save the Tweets in an array as a dictionery item 
#             Sentiment_array.append({"Influencers" : user,
#                                     "Tweet Text" : tweet["text"],
#                                     "Compound" : compound,
#                                     "Positive" : pos,
#                                     "Negative" : neg,
#                                     "Neutral" : neu,
#                                     "Date" : tweet["created_at"],
#                                     "Tweets Ago" : tweet_count
#                                    })
            
#             #Decrease count of tweet by 1 in the reverse order
#             tweet_count -= 1

# print("-----------End of Extraction of Tweets !!!-----------")


In [ ]:
# # Create dataframe from the Dictionery item of the Sentiment Array
# Sentiment_DF = pd.DataFrame.from_dict(Sentiment_array)

# # Remove the '@' from the 'influence' column in the data frame
# Sentiment_DF["Influencers"] = Sentiment_DF["Influencers"].map(lambda x: x.lstrip('@'))

# # Re_arrang the columns and save into a CSV file
# Sentiment_DF = Sentiment_DF[["Influencers", "Date", "Tweet Text"
#                              , "Compound", "Positive", "Negative"
#                              , "Neutral", "Tweets Ago"
#                             ]]

# # Store output in a .CSV File
# Sentiment_DF.to_csv("influencer_tweets_Analysis.csv")

# # Sentiment_DF

In [ ]:
# # Calculate the mean for each Influencers & store into a dataframe
# Influencers_Comp_Mean = Sentiment_DF.groupby("Influencers").mean()["Compound"].to_frame()

# #Reset the index 
# Influencers_Comp_Mean.reset_index(inplace=True)

# Influencers_Comp_Mean

In [ ]:
# # Set the values for x_axis & y_axis
# x_axis = Influencers_Comp_Mean.index.values
# y_axis = Influencers_Comp_Mean["Compound"]
# X_Label = ["@Oprah","@KimKardashian","@realDonaldTrump","@justinbieber","@KylieJenner"]

# # Intialize the plots. 
# fig,ax = plt.subplots()#  function that returns a tuple containing a figure and axes object(s)

# #Set the plot and assign the values like colors etc
# bars = ax.bar(x_axis,y_axis
#               , align = "edge"
#               , width = 1
#               , linewidth = 1
#               , edgecolor = 'black'
#               , color = ["yellow","lime","red","orange","pink"]
#              )

# # Set the tick(s) of the bar graph
# tick_locations = [value + 0.5 for value in range(len(x_axis))]
# plt.xticks(tick_locations,X_Label,rotation='vertical')

# # If value is positive then put True in the Summary else place False
# Influencers_Comp_Mean["Positive"] = Influencers_Comp_Mean["Compound"] > 0

# # Assign the height based on positive value after allocating True / false value
# height = Influencers_Comp_Mean.Positive.map({True: 0.03 , False: -0.03})

# # # Set the value on labels on the bars
# for bar in bars:
#     ax.text(bar.get_x() + bar.get_width()/2, bar.get_height() + height[bars.index(bar)]
#             , round(Influencers_Comp_Mean["Compound"][bars.index(bar)],3)
#             , ha = 'center'
#             , va = 'bottom'
#             )

# # Set the x_axis limits
# ax.set_xlim(0, len(x_axis))

# # Dynamically set the y_axis limits by finding the max & min value of y-axis
# ax.set_ylim(min(y_axis)-0.1, max(y_axis) + 0.1)

# # Set a horizontal line at y = 0
# plt.hlines(0,0,len(x_axis))

# # Title of the graph
# ax.set_title("Sentiments on Twitter of Influencers (%s)" % (time.strftime("%x")), fontsize=16)

# # Setting the y_axis label
# ax.set_ylabel("Polarity on Twitter ", fontsize=14)

# # # Setting the x_axis label
# ax.set_xlabel("The Influencers", fontsize=14)
  
# # Saving the graph
# plt.savefig("The Influencer Twitter Sentiment .png",bbox_inches='tight')
# plt.show()

In [ ]:
# # Create an array of Influencers Houses with the unique function in the data frame
# Influencers_array = Sentiment_DF["Influencers"].unique()
# Influencers_array

# # #Plotting the graph for each influencer
# for influencer in Influencers_array:
# # Creating a temporary data frame to store for only one influencer at a time
#         Temp_DF = Sentiment_DF[Sentiment_DF["influencer"] == influencer]
        
#         Sentiment_DF['influencer'] = Sentiment_DF['influencer'].map(lambda x: x.lstrip('@'))
# #Temp_DF
    
#         plt.scatter(Temp_DF["Tweets Ago"],Temp_DF["Compound"]
#                  , marker = "o", linewidth = 0, alpha = 0.8, label = Influencers
#                  , facecolors = Temp_DF.influencer.map({"@Oprah": "blue"
#                                                    , "@KimKardashian" : "lime"
#                                                    , "@realDonaldTrump": 'indigo'
#                                                    , "@justinbieber":"fuchsia"
#                                                    , "@KylieJenner":"gold"
#                                                   })
#                 )

# # # Set the legend 
# plt.legend(bbox_to_anchor = (1,1), title="The Influencers", loc='best')

# # # Set the labels of x_axis, y_axis & title 
# plt.xlabel("Tweets Ago", fontsize=12)
# plt.ylabel("Tweet Polarity", fontsize=12)
# plt.title("Sentiment Analysis of The Influencers Tweets (%s)" % (time.strftime("%x")), fontsize=16)

# # #Set the limite of  x_axis and y_axis
# plt.xlim(0, 101)
# plt.ylim(-1,1)

# # # Set the grid
# plt.grid(True)

# filePath = 'Images'
# if not path.exists(filePath):
#     makedirs(filePath)

# # Save the result to a .png file
# plt.savefig("Sentiment Analysis of Influencers Tweets.png",bbox_inches='tight')
# # plt.savefig("Sentiment Analysis of The influencer's Tweets.png",bbox_inches='tight')

# plt.show()

In [ ]:
# Version: 1.0
# Date: Fri June 29, 2018 
# Time: 3:50 PM
# Functionalities: 
# A) Data Analysis
#     1. Created dataframe.
#     2. Sorting
# B) Sentiment Analysis


# Pending items:
# Data Cleaning